# TESTES DE HIPÓTESES SOBRE UMA MÉDIA POPULACIONAL(µ)

# Teste Z para a média populacional (amostra aleatória)

Este teste verifica se a média de uma população ($\mu$) assume um valor específico ($\mu_0$) a partir de uma amostra aleatória. É apropriado quando:

- A população é (aproximadamente) normal OU a amostra é grande ($n>30$, pelo Teorema do Limite Central);
- O desvio-padrão populacional é conhecido ($\sigma$ conhecido).

> Se $\sigma$ for desconhecido e $n$ for pequeno, use o teste t de Student em vez do teste Z.

## Hipóteses
- Teste bilateral:  
  $H_0: \mu = \mu_0$  
  $H_1: \mu \neq \mu_0$

- Teste unilateral à direita:  
  $H_0: \mu \le \mu_0$  
  $H_1: \mu > \mu_0$

- Teste unilateral à esquerda:  
  $H_0: \mu \ge \mu_0$  
  $H_1: \mu < \mu_0$

## Estatística de teste
Padronize a média amostral $\bar X$ para compará-la com a distribuição normal padrão:

$$
Z_{calc} = \frac{\bar X - \mu_0}{\sigma_{\bar X}} \sim \mathcal N(0,1), \quad \text{onde} \quad \sigma_{\bar X} = \frac{\sigma}{\sqrt{n}}
$$

## Regra de decisão (região crítica)
Seja $\alpha$ o nível de significância:
- Bilateral: rejeite $H_0$ se $|Z_{calc}| > z_{\alpha/2}$.
- Unilateral à direita: rejeite $H_0$ se $Z_{calc} > z_{\alpha}$.
- Unilateral à esquerda: rejeite $H_0$ se $Z_{calc} < -z_{\alpha}$.

## p-valor
- Bilateral: $p = 2\,P\big(Z \ge |Z_{calc}|\big)$.
- Unilateral à direita: $p = P\big(Z \ge Z_{calc}\big)$.
- Unilateral à esquerda: $p = P\big(Z \le Z_{calc}\big)$.

Rejeite $H_0$ se $p \le \alpha$; caso contrário, não rejeite $H_0$.

## Função Teste Z para uma amostra

In [ ]:
# Bibliotecas
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns
from scipy.stats import norm
import pandas as pd
import scipy.stats as stats

In [ ]:
def teste_z_uma_amostra(
    x=None,              # dados brutos da amostra (iterável)
    n=None,              # tamanho da amostra, se não houver x
    xbar=None,           # média amostral, se não houver x
    sigma=None,          # desvio-padrão populacional (conhecido, > 0)
    mu0=0.0,             # valor hipotético de média sob H0
    alpha=0.05,          # nível de significância
    alternativa="bilateral"  # 'bilateral' | 'esquerda' | 'direita'
):
    alternativa = str(alternativa).lower()
    assert alternativa in ("bilateral", "esquerda", "direita"), "alternativa inválida"

    # Definição de n e xbar
    if x is not None:
        arr = np.asarray(list(x), dtype=float)
        n = int(arr.size)
        assert n > 0, "Amostra vazia."
        xbar = float(arr.mean())
    else:
        assert (n is not None) and (xbar is not None), "Forneça x ou (n e xbar)."
        n = int(n)
        xbar = float(xbar)
        assert n > 0, "n deve ser positivo."

    # Checagens
    assert sigma is not None and sigma > 0, "sigma (populacional) deve ser > 0"
    sigma = float(sigma)

    # Erro-padrão e estatística Z
    se = sigma / math.sqrt(n)
    Z_calc = (xbar - mu0) / se

    # p-valor conforme alternativa
    if alternativa == "bilateral":
        p_valor = 2.0 * min(stats.norm.cdf(Z_calc), 1.0 - stats.norm.cdf(Z_calc))
        z_c = stats.norm.ppf(1 - alpha/2)
        z_crit_esq, z_crit_dir = -z_c, z_c
        rejeita = (abs(Z_calc) > z_c) or (p_valor <= alpha)
    elif alternativa == "esquerda":
        p_valor = stats.norm.cdf(Z_calc)  # P(Z ≤ Z_calc)
        z_crit_esq = stats.norm.ppf(alpha)  # z_{α}
        z_crit_dir = None
        rejeita = (Z_calc < z_crit_esq) or (p_valor <= alpha)
    else:  # "direita"
        p_valor = 1.0 - stats.norm.cdf(Z_calc)  # P(Z ≥ Z_calc)
        z_crit_dir = stats.norm.ppf(1 - alpha)  # z_{1-α}
        z_crit_esq = None
        rejeita = (Z_calc > z_crit_dir) or (p_valor <= alpha)

    decisao = "Rejeitar" if rejeita else "Não rejeitar"

    return {
        "n": n,
        "xbar": xbar,
        "sigma": sigma,
        "mu0": mu0,
        "se": se,
        "Z_calc": float(Z_calc),
        "p_valor": float(p_valor),
        "alpha": float(alpha),
        "alternativa": alternativa,
        "z_crit_esq": None if z_crit_esq is None else float(z_crit_esq),
        "z_crit_dir": None if z_crit_dir is None else float(z_crit_dir),
        "decisao": decisao,
    }

## Função de plotagem Teste Z

In [ ]:
def plot_normal_z_test(Z_calc, alpha=0.05, alternativa="bilateral", titulo="Distribuição Normal Padrão - Teste Z"):
    alternativa = str(alternativa).lower()
    assert alternativa in ("bilateral", "esquerda", "direita"), "alternativa inválida"

    x = np.linspace(-4.5, 4.5, 1600)
    y = stats.norm.pdf(x)

    plt.figure(figsize=(9, 5))
    plt.plot(x, y, color="black", lw=1.8, label="N(0,1)")

    # Regiões críticas
    if alternativa == "bilateral":
        z_c = stats.norm.ppf(1 - alpha/2)
        mask_left_c = x <= -z_c
        mask_right_c = x >= z_c
        plt.fill_between(x[mask_left_c], 0, y[mask_left_c], color="tab:red", alpha=0.25, label=f"Regiões críticas (α={alpha:.2%})")
        plt.fill_between(x[mask_right_c], 0, y[mask_right_c], color="tab:red", alpha=0.25)
        plt.axvline(-z_c, color="tab:red", ls="--", lw=2, label=f"± z_c = {z_c:.3f}")
        plt.axvline(z_c, color="tab:red", ls="--", lw=2)

        # p-valor (bicaudal)
        z_abs = abs(Z_calc)
        mask_left_p = x <= -z_abs
        mask_right_p = x >= z_abs
        p_valor = 2 * (1 - stats.norm.cdf(z_abs))
        plt.fill_between(x[mask_left_p], 0, y[mask_left_p], color="tab:blue", alpha=0.35, label=f"p-valor = {p_valor:.4f}")
        plt.fill_between(x[mask_right_p], 0, y[mask_right_p], color="tab:blue", alpha=0.35)

    elif alternativa == "esquerda":
        z_c_esq = stats.norm.ppf(alpha)
        mask_alpha = x <= z_c_esq
        plt.fill_between(x[mask_alpha], 0, y[mask_alpha], color="tab:red", alpha=0.25, label=f"Região crítica (α={alpha:.2%})")
        plt.axvline(z_c_esq, color="tab:red", ls="--", lw=2, label=f"z_c = {z_c_esq:.3f}")

        # p-valor (cauda esquerda)
        mask_p = x <= Z_calc
        p_valor = stats.norm.cdf(Z_calc)
        plt.fill_between(x[mask_p], 0, y[mask_p], color="tab:blue", alpha=0.35, label=f"p-valor = {p_valor:.4f}")

    else:  # "direita"
        z_c_dir = stats.norm.ppf(1 - alpha)
        mask_alpha = x >= z_c_dir
        plt.fill_between(x[mask_alpha], 0, y[mask_alpha], color="tab:red", alpha=0.25, label=f"Região crítica (α={alpha:.2%})")
        plt.axvline(z_c_dir, color="tab:red", ls="--", lw=2, label=f"z_c = {z_c_dir:.3f}")

        # p-valor (cauda direita)
        mask_p = x >= Z_calc
        p_valor = 1 - stats.norm.cdf(Z_calc)
        plt.fill_between(x[mask_p], 0, y[mask_p], color="tab:blue", alpha=0.35, label=f"p-valor = {p_valor:.4f}")

    # Linha do Z_calc
    plt.axvline(Z_calc, color="tab:blue", ls="-", lw=2, label=f"Z_calc = {Z_calc:.3f}")

    plt.title(titulo)
    plt.xlabel("z")
    plt.ylabel("densidade")
    plt.legend(loc="upper right")
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()

# Teste Z para uma amostra
### (Exemplo 8 do Material)

Contexto: o fabricante afirma μ ≥ 4,2 g (σ conhecido = 1 g).  
A amostra (n = 42) apresentou média x̄ = 3,9 g. Nível de significância α = 5%.

Hipóteses (unilateral à esquerda):
- H0: μ = 4,2
- H1: μ < 4,2

Regra: rejeitar H0 se z_calc < z_{α} ou se p-valor < α.

In [ ]:
# Exemplo 8 - Cálculo do teste Z (unilateral à esquerda)
alpha = 0.05
mu0 = 4.2
sigma = 1.0
n = 42
xbar = 3.9

# Usa a função nova de teste Z para uma amostra
res8 = teste_z_uma_amostra(n=n, xbar=xbar, sigma=sigma, mu0=mu0, alpha=alpha, alternativa="esquerda")

# Impressão organizada dos resultados
print(
    f"n = {res8['n']}, xbar = {res8['xbar']:.3f}, σ = {res8['sigma']:.3f}, μ0 = {res8['mu0']:.3f}\n"
    f"SE = {res8['se']:.4f}, Z_calc = {res8['Z_calc']:.4f}, z_c = {res8['z_crit_esq']:.4f}, p-valor = {res8['p_valor']:.4f}\n"
    f"Decisão: {res8['decisao']} (α = {res8['alpha']:.2%}, alternativa = {res8['alternativa']})"
)

# Variáveis para o gráfico
Z_calc_8 = float(res8['Z_calc'])
alpha_8 = float(res8['alpha'])
alt_8 = res8['alternativa']

z_calc = -1.9442
z_c_esq = -1.6449
p-valor = 0.0259
Decisão: Rejeitar H0


## Plotagem do gráfico

In [ ]:
plot_normal_z_test(Z_calc_8, alpha=alpha_8, alternativa=alt_8, titulo="Teste Z - unilateral à esquerda")

## Teste Z com Scipy

In [ ]:
# Teste Z (mesmo exemplo) usando SciPy
alpha = 0.05
mu0 = 4.2
sigma = 1.0
n = 42
xbar = 3.9

# Cálculos com SciPy
se = sigma / math.sqrt(n)
Z_calc = (xbar - mu0) / se
z_crit_esq = stats.norm.ppf(alpha)      # z_{α} (cauda esquerda)
p_valor = stats.norm.cdf(Z_calc)        # P(Z <= Z_calc)

decisao_scipy = "Rejeitar" if (Z_calc < z_crit_esq) or (p_valor <= alpha) else "Não rejeitar"

print(
    f"n = {n}, xbar = {xbar:.3f}, σ = {sigma:.3f}, μ0 = {mu0:.3f}\n"
    f"SE = {se:.4f}, Z_calc = {Z_calc:.4f}, z_c = {z_crit_esq:.4f}, p-valor = {p_valor:.4f}\n"
    f"Decisão: {decisao_scipy} (α = {alpha:.2%}, alternativa = 'esquerda')"
)

# Comparação Manual × SciPy (rever em casa para remover o comeco)

In [ ]:
# Teste t (Exemplo 9) usando SciPy e comparação Manual × SciPy
# SciPy
T_scipy_9 = (xbar_9 - mu0_9) / (s_9 / math.sqrt(n_9))
p_scipy_9 = stats.t.cdf(T_scipy_9, df=n_9-1)  # cauda esquerda

tcrit_esq_9 = stats.t.ppf(alpha_9, df=n_9-1)
decisao_scipy_9 = "Rejeitar H0" if (T_scipy_9 < tcrit_esq_9) or (p_scipy_9 <= alpha_9) else "Não rejeitar H0"

# Tabela comparativa
import pandas as _pd
comp9 = _pd.DataFrame({
    "Origem": ["Manual", "SciPy"],
    "T_calc": [T_calc_9, T_scipy_9],
    "p_valor": [res9['p_valor'], p_scipy_9],
    "t_crit_esq": [res9['t_crit_esq'], tcrit_esq_9],
    "Decisão": [res9['decisao'], decisao_scipy_9]
})
print(comp9.to_string(index=False, justify='center', float_format=lambda v: f"{v:.4f}"))

# Gráfico comparativo
x = np.linspace(stats.t.ppf(0.001, gl_9), stats.t.ppf(0.999, gl_9), 1200)
y = stats.t.pdf(x, gl_9)

fig, ax = plt.subplots(1, 2, figsize=(12, 5), sharey=True)

# Manual
mask_c_m = x <= res9['t_crit_esq']
mask_p_m = x <= T_calc_9
ax[0].plot(x, y, color='black', lw=1.5)
ax[0].fill_between(x[mask_c_m], 0, y[mask_c_m], color='tab:red', alpha=0.25, label=f"Região crítica (α={alpha_9:.2%})")
ax[0].fill_between(x[mask_p_m], 0, y[mask_p_m], color='tab:blue', alpha=0.35, label=f"p-valor = {res9['p_valor']:.4f}")
ax[0].axvline(res9['t_crit_esq'], color='tab:red', ls='--', lw=2, label=f"t_c = {res9['t_crit_esq']:.3f}")
ax[0].axvline(T_calc_9, color='tab:blue', lw=2, label=f"T_calc = {T_calc_9:.3f}")
ax[0].set_title('Manual')
ax[0].set_xlabel('t')
ax[0].set_ylabel('densidade')
ax[0].grid(alpha=0.3)
ax[0].legend(loc='upper right')

# SciPy
mask_c_s = x <= tcrit_esq_9
mask_p_s = x <= T_scipy_9
ax[1].plot(x, y, color='black', lw=1.5)
ax[1].fill_between(x[mask_c_s], 0, y[mask_c_s], color='tab:red', alpha=0.25, label=f"Região crítica (α={alpha_9:.2%})")
ax[1].fill_between(x[mask_p_s], 0, y[mask_p_s], color='tab:blue', alpha=0.35, label=f"p-valor = {p_scipy_9:.4f}")
ax[1].axvline(tcrit_esq_9, color='tab:red', ls='--', lw=2, label=f"t_c = {tcrit_esq_9:.3f}")
ax[1].axvline(T_scipy_9, color='tab:blue', lw=2, label=f"T_calc = {T_scipy_9:.3f}")
ax[1].set_title('SciPy')
ax[1].set_xlabel('t')
ax[1].grid(alpha=0.3)
ax[1].legend(loc='upper right')

fig.suptitle('Exemplo 9 — Comparação Manual × SciPy (t unilateral à esquerda)')
plt.tight_layout()
plt.show()

## Teste t de Student (uma amostra)

Uso: quando o desvio-padrão populacional (σ) é desconhecido. Nessa situação, σ é estimado pelo desvio-padrão amostral S e a estatística de teste passa a seguir a distribuição t de Student com n − 1 graus de liberdade. É apropriado quando a população é aproximadamente normal ou quando a amostra não é grande o suficiente para invocar o TLC com segurança.

Hipóteses típicas
- Teste bilateral:
  - H0: μ = μ0
  - H1: μ ≠ μ0
- Teste unilateral à direita:
  - H0: μ ≤ μ0
  - H1: μ > μ0
- Teste unilateral à esquerda:
  - H0: μ ≥ μ0
  - H1: μ < μ0

Estatística de teste
A média amostral X̄ é padronizada usando S/√n em vez de σ/√n:

$$
T_{calc} = \frac{\bar X - \mu_0}{S/\sqrt{n}} \;\sim\; t_{\,n-1}
$$

Regra de decisão (regiões críticas)

Seja α = significância e t_{α,ν} = quantil da t de Student com ν = n−1 gl.
- Bilateral: rejeitar H0 se |T_calc| > t_{α/2,\,n-1}.
- Unilateral à direita: rejeitar H0 se T_calc > t_{α,\,n-1}.
- Unilateral à esquerda: rejeitar H0 se T_calc < −t_{α,\,n-1}.

p-valor
Considere T ∼ t_{n−1}:
- Bilateral: p = 2·P(T ≥ |T_calc|).
- Direita: p = P(T ≥ T_calc).
- Esquerda: p = P(T ≤ T_calc).
Decida por p-valor: rejeite H0 se p ≤ α; caso contrário, não rejeite.

### Função Manual

In [ ]:
def teste_t_uma_amostra_manual(
    x=None,                 # dados brutos (iterável)
    n=None,                 # tamanho da amostra, se não houver x
    xbar=None,              # média amostral, se não houver x
    s=None,                 # desvio-padrão amostral (> 0)
    mu0=0.0,                # média hipotética sob H0
    alpha=0.05,             # nível de significância
    alternativa="bilateral" # 'bilateral' | 'esquerda' | 'direita'
):
    alternativa = str(alternativa).lower()
    assert alternativa in ("bilateral", "esquerda", "direita"), "alternativa inválida"

    # Determinação de n, xbar e s quando x é fornecido
    if x is not None:
        arr = np.asarray(list(x), dtype=float)
        n = int(arr.size)
        assert n > 1, "É necessário n > 1 para calcular o desvio-padrão amostral."
        xbar = float(arr.mean())
        s = float(arr.std(ddof=1))
    else:
        # Resumo (n, xbar, s) obrigatórios
        assert (n is not None) and (xbar is not None) and (s is not None), "Forneça x ou (n, xbar, s)."
        n = int(n)
        xbar = float(xbar)
        s = float(s)
        assert n > 1, "n deve ser > 1"
        assert s > 0, "s (desvio-padrão amostral) deve ser > 0"

    gl = n - 1
    se = s / math.sqrt(n)
    T_calc = (xbar - mu0) / se

    # p-valor e valores críticos conforme alternativa
    if alternativa == "bilateral":
        p_valor = 2.0 * min(stats.t.cdf(T_calc, df=gl), 1.0 - stats.t.cdf(T_calc, df=gl))
        t_c = stats.t.ppf(1 - alpha/2, df=gl)
        t_crit_esq, t_crit_dir = -t_c, t_c
        rejeita = (abs(T_calc) > t_c) or (p_valor <= alpha)
    elif alternativa == "esquerda":
        p_valor = stats.t.cdf(T_calc, df=gl)            # P(T ≤ T_calc)
        t_crit_esq = stats.t.ppf(alpha, df=gl)         # t_{α,gl}
        t_crit_dir = None
        rejeita = (T_calc < t_crit_esq) or (p_valor <= alpha)
    else:  # "direita"
        p_valor = 1.0 - stats.t.cdf(T_calc, df=gl)     # P(T ≥ T_calc)
        t_crit_dir = stats.t.ppf(1 - alpha, df=gl)     # t_{1-α,gl}
        t_crit_esq = None
        rejeita = (T_calc > t_crit_dir) or (p_valor <= alpha)

    decisao = "Rejeitar" if rejeita else "Não rejeitar"

    return {
        "n": n,
        "xbar": xbar,
        "s": s,
        "se": se,
        "gl": int(gl),
        "T_calc": float(T_calc),
        "p_valor": float(p_valor),
        "alpha": float(alpha),
        "alternativa": alternativa,
        "t_crit_esq": None if t_crit_esq is None else float(t_crit_esq),
        "t_crit_dir": None if t_crit_dir is None else float(t_crit_dir),
        "decisao": decisao,
    }

### (Exemplo 9 do Material)

Problema: Antes, o tempo médio era μ0 = 18 min. Após mudanças, coletou-se uma amostra (n = 25) com média xbar = 16,808 min e desvio-padrão s = 2,733 min.  
Objetivo: Verificar, ao nível de significância α = 1%, se há evidência de melhoria (redução do tempo médio).

### Dados

Hipóteses (teste unilateral à esquerda):
- H0: μ = 18
- H1: μ < 18

- s = 2,733 min
- xbar = 16,808

### Teste Manual

In [ ]:
# Parâmetros do Exemplo 9 (unilateral à esquerda)
alpha_9 = 0.01
mu0_9 = 18.0
n_9 = 25
xbar_9 = 16.808
s_9 = 2.733

# Teste manual usando a função criada
res9 = teste_t_uma_amostra_manual(n=n_9, xbar=xbar_9, s=s_9, mu0=mu0_9, alpha=alpha_9, alternativa="esquerda")

print(
    f"n = {res9['n']}, xbar = {res9['xbar']:.3f}, s = {res9['s']:.3f}, μ0 = {mu0_9:.3f}\n"
    f"SE = {res9['se']:.4f}, gl = {res9['gl']}, T_calc = {res9['T_calc']:.4f}, t_c = {res9['t_crit_esq']:.4f}, p-valor = {res9['p_valor']:.6f}\n"
    f"Decisão: {res9['decisao']} (α = {res9['alpha']:.2%}, alternativa = {res9['alternativa']})"
)

# Conclusão contextual
if res9['decisao'].startswith('Rejeitar'):
    conclusao_9 = (
        "Há evidências, ao nível de 1%, de que a média reduziu em relação a 18 min (melhoria)."
    )
else:
    conclusao_9 = (
        "Não há evidências suficientes, ao nível de 1%, de que a média seja menor que 18 min (sem comprovar melhoria)."
    )
print("Conclusão:", conclusao_9)

# Variáveis para plotagem
T_calc_9 = float(res9['T_calc'])
gl_9 = int(res9['gl'])

T_calc = -2.1808, gl = 24, t_c_esq = -2.4922, p-valor = 0.0196
Decisão: Não rejeitar H0


In [ ]:
x = np.linspace(stats.t.ppf(0.001, gl_9), stats.t.ppf(0.999, gl_9), 1600)
y = stats.t.pdf(x, df=gl_9)

t_c_esq = float(res9['t_crit_esq'])
p_valor_9 = float(res9['p_valor'])

fig, ax = plt.subplots(figsize=(9, 5))
ax.plot(x, y, color='black', lw=1.8, label=f"t(df={gl_9})")

# Região crítica (α na cauda esquerda)
mask_crit = x <= t_c_esq
ax.fill_between(x[mask_crit], 0, y[mask_crit], color='tab:red', alpha=0.25,
                label=f"Região crítica (α={alpha_9:.2%})")
ax.axvline(t_c_esq, color='tab:red', ls='--', lw=2, label=f"t_c = {t_c_esq:.3f}")

# p-valor (área até T_calc na cauda esquerda)
mask_p = x <= T_calc_9
ax.fill_between(x[mask_p], 0, y[mask_p], color='tab:blue', alpha=0.35,
                label=f"p-valor = {p_valor_9:.4f}")
ax.axvline(T_calc_9, color='tab:blue', lw=2, label=f"T_calc = {T_calc_9:.3f}")

ax.set_title('Exemplo 9 — Teste t (unilateral à esquerda)')
ax.set_xlabel('t')
ax.set_ylabel('densidade')
ax.legend(loc='upper right')
ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()

### Teste t com Scipy

In [ ]:
# Estatística T e p-valor (cauda esquerda)
se_9 = s_9 / math.sqrt(n_9)
T_scipy_9 = (xbar_9 - mu0_9) / se_9
p_scipy_9 = stats.t.cdf(T_scipy_9, df=n_9 - 1)  # P(T <= T_calc)

# Valor crítico para α na cauda esquerda
tcrit_esq_9 = stats.t.ppf(alpha_9, df=n_9 - 1)

# Decisão (mesmo critério do manual)
decisao_scipy_9 = "Rejeitar" if (T_scipy_9 < tcrit_esq_9) or (p_scipy_9 <= alpha_9) else "Não rejeitar"

print(
    f"n = {n_9}, xbar = {xbar_9:.3f}, s = {s_9:.3f}, μ0 = {mu0_9:.3f}\n"
    f"SE = {se_9:.4f}, gl = {n_9-1}, T_calc = {T_scipy_9:.4f}, t_c = {tcrit_esq_9:.4f}, p-valor = {p_scipy_9:.6f}\n"
    f"Decisão: {decisao_scipy_9} (α = {alpha_9:.2%}, alternativa = 'esquerda')"
)

### Comparação Manual x Scipy

In [ ]:
_comp = pd.DataFrame({
    "Origem": ["Manual", "SciPy"],
    "T_calc": [T_calc_9, T_scipy_9],
    "p_valor": [res9['p_valor'], p_scipy_9],
    "t_crit_esq": [res9['t_crit_esq'], tcrit_esq_9],
    "Decisão": [res9['decisao'], decisao_scipy_9]
})
print(_comp.to_string(index=False, justify='center', float_format=lambda v: f"{v:.4f}"))

# Gráfico comparativo lado a lado
x = np.linspace(stats.t.ppf(0.001, gl_9), stats.t.ppf(0.999, gl_9), 1200)
y = stats.t.pdf(x, gl_9)

fig, ax = plt.subplots(1, 2, figsize=(12, 5), sharey=True)

# Manual
mask_c_m = x <= res9['t_crit_esq']
mask_p_m = x <= T_calc_9
ax[0].plot(x, y, color='black', lw=1.5)
ax[0].fill_between(x[mask_c_m], 0, y[mask_c_m], color='tab:red', alpha=0.25, label=f"Região crítica (α={alpha_9:.2%})")
ax[0].fill_between(x[mask_p_m], 0, y[mask_p_m], color='tab:blue', alpha=0.35, label=f"p-valor = {res9['p_valor']:.4f}")
ax[0].axvline(res9['t_crit_esq'], color='tab:red', ls='--', lw=2, label=f"t_c = {res9['t_crit_esq']:.3f}")
ax[0].axvline(T_calc_9, color='tab:blue', lw=2, label=f"T_calc = {T_calc_9:.3f}")
ax[0].set_title('Manual')
ax[0].set_xlabel('t')
ax[0].set_ylabel('densidade')
ax[0].grid(alpha=0.3)
ax[0].legend(loc='upper right')

# SciPy
mask_c_s = x <= tcrit_esq_9
mask_p_s = x <= T_scipy_9
ax[1].plot(x, y, color='black', lw=1.5)
ax[1].fill_between(x[mask_c_s], 0, y[mask_c_s], color='tab:red', alpha=0.25, label=f"Região crítica (α={alpha_9:.2%})")
ax[1].fill_between(x[mask_p_s], 0, y[mask_p_s], color='tab:blue', alpha=0.35, label=f"p-valor = {p_scipy_9:.4f}")
ax[1].axvline(tcrit_esq_9, color='tab:red', ls='--', lw=2, label=f"t_c = {tcrit_esq_9:.3f}")
ax[1].axvline(T_scipy_9, color='tab:blue', lw=2, label=f"T_calc = {T_scipy_9:.3f}")
ax[1].set_title('Scipy')
ax[1].set_xlabel('t')
ax[1].grid(alpha=0.3)
ax[1].legend(loc='upper right')

fig.suptitle('Comparação Manual × Scipy')
plt.tight_layout()
plt.show()

## Teste t de Student para duas amostras independentes

O teste t para duas amostras independentes é usado para comparar as médias de duas populações a partir de duas amostras aleatórias e independentes, quando os desvios‑padrão populacionais são desconhecidos.

Hipóteses (forma bilateral)
- H0: μ1 = μ2
- H1: μ1 ≠ μ2

A estatística de teste T depende de como tratamos as variâncias populacionais entre os grupos:

### CASO 1 — Variâncias diferentes (σ1² ≠ σ2²)
$$
T_{calc} = \frac{\bar X_1 - \bar X_2}{\sqrt{\dfrac{S_1^2}{n_1} + \dfrac{S_2^2}{n_2}}}
$$
Graus de liberdade:
$$
\nu = \frac{\left(\dfrac{S_1^2}{n_1} + \dfrac{S_2^2}{n_2}\right)^2}{\dfrac{\left(\dfrac{S_1^2}{n_1}\right)^2}{n_1-1} + \dfrac{\left(\dfrac{S_2^2}{n_2}\right)^2}{n_2-1}}
$$

### CASO 2 — Variâncias iguais (σ1² = σ2²)
$$
S_p^2 = \frac{(n_1-1)S_1^2 + (n_2-1)S_2^2}{n_1 + n_2 - 2}
$$
Estatística de teste:
$$
T_{calc} = \frac{\bar X_1 - \bar X_2}{S_p\,\sqrt{\dfrac{1}{n_1} + \dfrac{1}{n_2}}}
$$
Graus de liberdade: $\nu = n_1 + n_2 - 2$.

Regra de decisão (nível de significância α)
rejeite H0 se $|T_{calc}| > t_{\alpha/2,\,\nu}$.

p-valor
$p = 2\,P\big(T_{\nu} \ge |T_{calc}|\big)$.

### (Exemplo 10 do Material)

Um engenheiro de qualidade suspeita que o tempo médio de fabricação de um produto plástico depende do fornecedor da matéria‑prima. Para investigar, foram coletadas duas amostras independentes de tempos de fabricação: 30 observações do Fornecedor 1 e 30 do Fornecedor 2 (ver Tabelas 7.10 e 7.11 do material).

Objetivo
- Ao nível de significância α = 5%, verificar se há diferença entre as médias dos dois fornecedores (teste bilateral).

Formulação
- H0: μ1 = μ2 (as médias são iguais)
- H1: μ1 ≠ μ2 (as médias são diferentes)

### Teste Manual

### Teste com Scipy

### Comparação Manual x Scipy

## Teste t de Student para duas amostras emparelhadas

### (Exemplo 11 do Material)

In [58]:
antes  = [3.2, 3.6, 3.4, 3.8, 3.4, 3.5, 3.7, 3.2, 3.5, 3.9]
depois = [3.0, 3.3, 3.5, 3.6, 3.4, 3.3, 3.4, 3.0, 3.2, 3.6]

### Teste Manual

### Teste Scipy

### Comparação Manual x Scipy